In [ ]:
# optionally add scripts location to path
if True:
    import sys
    sys.path.append("/home/physics3/rroussel/SLAC_Xopt/")
    sys.path.append("/home/physics3/rroussel/SLAC_Xopt/NN_prior/")

print(sys.path)

In [1]:
import time
from typing import Dict

import torch
import numpy as np
import matplotlib.pyplot as plt

from scripts.evaluate_function.screen_image import measure_beamsize, measure_background
from scripts.optimize_function import optimize_function
from scripts.characterize_emittance import characterize_emittance

from xopt import Xopt, VOCS
from xopt.evaluator import Evaluator
from xopt.numerical_optimizer import LBFGSOptimizer
from xopt.generators import ExpectedImprovementGenerator
from xopt.generators.bayesian.models.standard import StandardModelConstructor
from lume_model.utils import variables_from_yaml
from lume_model.torch import LUMEModule, PyTorchModel

from custom_mean import CustomMean
from dynamic_custom_mean import DynamicCustomMean
from metric_informed_custom_mean import MetricInformedCustomMean

## Initialization

In [ ]:
# set up data saving locations
data_dir = "/home/physics3/ml_tuning/20230729_LCLS_Injector"

run_name = "be_1"
run_dir = f"{data_dir}/{run_name}"
import os
if not os.path.exists(run_dir):
    os.mkdir(run_dir)

In [ ]:
## import variable range
import pandas as pd
filename = "../../variables.csv"
VARIABLE_RANGES = pd.read_csv(filename, index_col=0, header=None).T.to_dict(orient='list')
SCREEN_NAME = "OTRS:IN20:621" # OTR 3

In [ ]:
# set up background
BACKGROUND_FILE = f"{data_dir}/{SCREEN_NAME}_background.npy".replace(":","_")

## define evaluate function

In [ ]:
from epics import caget_many
import json
secondary_observables = ["SOLN:IN20:121:BACT","QUAD:IN20:121:BACT","QUAD:IN20:122:BACT","QUAD:IN20:361:BACT","QUAD:IN20:371:BACT","QUAD:IN20:425:BACT","QUAD:IN20:441:BACT","QUAD:IN20:511:BACT","QUAD:IN20:525:BACT","SOLN:IN20:121:BCTRL","QUAD:IN20:121:BCTRL","QUAD:IN20:122:BCTRL","QUAD:IN20:361:BCTRL","QUAD:IN20:371:BCTRL","QUAD:IN20:425:BCTRL","QUAD:IN20:441:BCTRL","QUAD:IN20:511:BCTRL","QUAD:IN20:525:BCTRL","IRIS:LR20:130:CONFG_SEL","ACCL:IN20:300:L0A_PDES","ACCL:IN20:300:L0A_ADES","ACCL:IN20:400:L0B_PDES","ACCL:IN20:400:L0B_ADES","ACCL:IN20:300:L0A_S_PV","ACCL:IN20:400:L0B_S_PV","ACCL:IN20:300:L0A_S_AV","ACCL:IN20:400:L0B_S_AV","GUN:IN20:1:GN1_ADES","GUN:IN20:1:GN1_S_AV","GUN:IN20:1:GN1_S_PV","LASR:IN20:196:PWR1H","LASR:IN20:475:PWR1H","SIOC:SYS0:ML01:CALCOUT008","REFS:IN20:751:EDES","CAMR:IN20:186:ZERNIKE_COEFF","FBCK:BCI0:1:CHRG_S","PMTR:LR20:121:PWR","BPMS:IN20:731:X","TCAV:IN20:490:TC0_C_1_TCTL","KLYS:LI20:51:BEAMCODE1_TCTL","LASR:LR20:1:UV_LASER_MODE","LASR:LR20:1:IR_LASER_MODE","IOC:BSY0:MP01:LSHUTCTL","WPLT:LR20:220:LHWP_ANGLE","OTRS:IN20:621:PNEUMATIC"]

# define function to measure the total size on OTR4
def eval_beamsize(input_dict):
    results = measure_beamsize(input_dict)

    # get secondary PV settings/readbacks
    secondary_results = caget_many(secondary_observables)
    results = results | secondary_results


    results["S_x_mm"] = results["Sx"] * 1e3
    results["S_y_mm"] = results["Sy"] * 1e3

    #add total beam size
    results["total_size"] = np.sqrt(results["Sx"]**2 + results["Sy"]**2)
    return results

## Create model

## Build vocs

In [1]:
# inputs
TUNING_VARIABLES = [
    "SOLN:IN20:121:BCTRL",
    "QUAD:IN20:121:BCTRL",
    "QUAD:IN20:122:BCTRL",
    "QUAD:IN20:361:BCTRL",
    "QUAD:IN20:371:BCTRL",
    "QUAD:IN20:425:BCTRL",
    "QUAD:IN20:441:BCTRL",
    "QUAD:IN20:511:BCTRL",
    "QUAD:IN20:525:BCTRL"
]

ROI = None
THRESHOLD = 3

measurement_options = {
    "screen": SCREEN_NAME,
    "background": BACKGROUND_FILE,
    "threshold": THRESHOLD,
    "roi": ROI,
    "bb_half_width": 2.0, # half width of the bounding box in terms of std
    "visualize": False,
    "save_img_location": run_dir,
    "sleep_time": 3.0,
    "n_shots": 5
}

image_constraints = {
    "bb_penalty": ["LESS_THAN", 0.0],
    "log10_total_intensity": ["GREATER_THAN", 4]
}

vocs = VOCS(
    variables={ele: VARIABLE_RANGES[ele] for ele in TUNING_VARIABLES},
    constants=measurement_options,
    objectives={"total_beamsize": "MINIMIZE"},
    constraints=image_constraints,
)

SyntaxError: invalid syntax (1400939419.py, line 13)

In [11]:
print(vocs.as_yaml())

variables:
  SOLN:IN20:121:BACT: [0.3774080152672698, 0.4983800018349345]
  QUAD:IN20:121:BACT: [-0.02098429469554406, 0.020999198106589838]
  QUAD:IN20:122:BACT: [-0.020998830517503037, 0.020998929132148195]
  QUAD:IN20:361:BACT: [-4.318053641915576, -1.0800430432494976]
  QUAD:IN20:371:BACT: [1.0913525514575348, 4.30967984810423]
  QUAD:IN20:425:BACT: [-7.559759590824369, -1.080762695815712]
  QUAD:IN20:441:BACT: [-1.0782202690353522, 7.559878303179915]
  QUAD:IN20:511:BACT: [-1.0792451325247663, 7.5582919025608595]
  QUAD:IN20:525:BACT: [-7.557932980106783, -1.0800286565992732]
constraints: {}
objectives: {sigma_xy: MINIMIZE}
constants: {'CAMR:IN20:186:R_DIST': 423.867825, Pulse_length: 1.8550514181818183,
  'FBCK:BCI0:1:CHRG_S': 0.25, 'ACCL:IN20:300:L0A_ADES': 58.0, 'ACCL:IN20:300:L0A_PDES': -9.53597349,
  'ACCL:IN20:400:L0B_ADES': 70.0, 'ACCL:IN20:400:L0B_PDES': 9.85566222}
observables: ['OTRS:IN20:571:XRMS', 'OTRS:IN20:571:YRMS']



## Define prior mean

In [14]:
# define custom mean
# wrap in LUMEModule
lume_module = LUMEModule(
    model=lume_model,
    feature_order=vocs.variable_names,
    output_order=lume_model.outputs[0:2],
)

# define objective model
class ObjectiveModel(torch.nn.Module):
    def __init__(self, model: LUMEModule):
        super(ObjectiveModel, self).__init__()
        self.model = model

    @staticmethod
    def function(sigma_x: torch.Tensor, sigma_y: torch.Tensor) -> torch.Tensor:
        # using this calculation due to occasional negative values
        return torch.sqrt(sigma_x ** 2 + sigma_y ** 2)

    def forward(self, x) -> torch.Tensor:
        idx_sigma_x = self.model.output_order.index("OTRS:IN20:571:XRMS")
        idx_sigma_y = self.model.output_order.index("OTRS:IN20:571:YRMS")
        sigma_x = self.model(x)[..., idx_sigma_x]
        sigma_y = self.model(x)[..., idx_sigma_y]
        return self.function(sigma_x, sigma_y)


objective_model = ObjectiveModel(lume_module)

print(lume_module.input_order)
print(lume_module.output_order)

mean_class = CustomMean
mean_kwargs = {"model": objective_model}

## Create Xopt instance

In [15]:
# Xopt definition
model_constructor = StandardModelConstructor(
    mean_modules={"total_beamsize": mean_class(**mean_kwargs)},
    trainable_mean_keys=[vocs.objective_names[0]],
)
generator = ExpectedImprovementGenerator(
    vocs=vocs,
    model_constructor=model_constructor,
)
evaluator = Evaluator(function=eval_beamsize)
X = Xopt(generator=generator, evaluator=evaluator, vocs=vocs)

## Create initial samples

In [ ]:
# create initial samples
n_init = 3
X.random_evaluate(n_samples=n_init)

## Run optimization

In [ ]:
n_step = 50
for step in range(n_step):
    # define prior mean
    if issubclass(mean_class, DynamicCustomMean):
        mean_kwargs["step"] = step
    elif issubclass(mean_class, MetricInformedCustomMean):
        if step == 0:
            mean_kwargs["metrics"] = {"correlation": 1.0}
        else:
            x_samples = torch.tensor(X.data[vocs.variable_names].values, dtype=torch.double)
            y_samples = torch.tensor(X.data[vocs.objective_names].values, dtype=torch.double).squeeze()
            y_samples_prior = mean_kwargs["model"](x_samples).squeeze()
            correlation = calc_corr(y_samples, y_samples_prior)
            mean_kwargs["metrics"] = {"correlation": correlation}
    X.generator.model_constructor.mean_modules[vocs.objective_names[0]] = mean_class(**mean_kwargs)

    # optimization step
    t0 = time.time()
    X.step()
    print("Completed step {:d} ({:.2f} sec)".format(step, time.time() - t0))